In [2]:
# module set
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from collections import OrderedDict
from itertools import repeat

In [3]:
import os
os.chdir('C:/Users/Yeeun/OneDrive/Desktop/Parrot/2020/aaaab')

In [4]:
driver = webdriver.Chrome("drivers/chromedriver")
driver.implicitly_wait(3)
player_url = 'http://fifaonline4.inven.co.kr/dataninfo/player/?code='

### **Players' DB** - by Player position

In [5]:
def position_db(pos):
    # 선수 DB 페이지로 이동

    driver.get('http://fifaonline4.inven.co.kr/dataninfo/player/')
    driver.implicitly_wait(5)
    # 포지션 체크// checkbox의 xpath가 아닌 span 태그의 xpath를 긁어와야 함.
    if pos == 'st':
        checkbox = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[1]/li[3]/div[2]/div[1]/div[2]/label[1]/span')
        checkbox.click()
    if pos == 'mid':
        checkbox = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[1]/li[3]/div[2]/div[2]/div[2]/label[1]/span')
        checkbox.click()
    if pos == 'def':
        checkbox = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[1]/li[3]/div[2]/div[3]/div[2]/label[1]/span')
        checkbox.click()
    if pos =='gl':
        checkbox = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[1]/li[3]/div[2]/div[4]/div[2]/label[1]/span')
        checkbox.click()
    search = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[2]/li[17]/button[1]/img')
    search.click()

### **Players DB by season(card class)**

In [6]:
seasons = [['LH','TKL','20KL','19S', '19UCL', '20TOTY','20TOTY-N'],
          ['NG', 'COC', 'OTW', '19A', '19', 'HOT', '19TOTS'],
          ['ICON', 'TC', '18S', 'MC-ICON', '18TOTY', 'GR', '19KFA'],
          ['18A', 'TT', 'TB', 'MCFC', '18KFA', 'NHD']]

In [7]:
def click_by_class(x, y):
    driver.get('http://fifaonline4.inven.co.kr/dataninfo/player/')
    driver.implicitly_wait(5)
#     //*[@id="fifaonline4Db"]/div[2]/ul[1]/li[2]/div[2]/div[2]/label[1]/span
    temp_path = f'//*[@id="fifaonline4Db"]/div[2]/ul[1]/li[2]/div[2]/div[{x}]/label[{y}]/span'
    button = driver.find_element_by_xpath(temp_path).click()
    search = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[2]/li[17]/button[1]/img')
    search.click()

In [8]:
from tqdm.auto import tqdm

### Get players information from DB page


In [9]:
def players_info(html):
    soup = bs(html, 'html.parser')
    search_url_list = []
    
    for link in soup.find_all('a'):
        if 'href' in link.attrs:
            li = link.attrs['href'].split('\n')
            if li[0].startswith(player_url):
                search_url_list.append(li[0])
            else:
                continue
    names = [name.text for name in soup.select('span > b')]
    search_url_list = list(OrderedDict(zip(search_url_list, repeat(None))))
    print("num of url : ", len(search_url_list))
    print(search_url_list[:5])
#     print(search_url_list)
    return search_url_list

In [10]:
def StatDB(url_list):
    total_df=pd.DataFrame()
    
    for search_url in tqdm(url_list): # 아까 검색해야할 url 리스트에서 하나씩 가져온다
        one_player_page = requests.get(search_url)
        one_soup=bs(one_player_page.content,'lxml')
        # one_player_db 는 각 선수마다 데이터베이스. 루프 한번 돌 때마다 각 선수의 DB를 일단 긁어옴
        one_player_db=one_soup.find_all("div", {"class":"fifa4 db_tooltip"})
    
        for db in one_player_db:
            detail_dict={}  
            for i,p in enumerate(db.find_all("p")):
                if i<10:
                    if i==0:
                        detail_dict['name']=p.text
                    elif i==7:
                        detail_dict['foot_left']=p.text
                    elif i==8:
                        detail_dict['foot_right_act']=p.text
                    elif i==9:
                        detail_dict['specialty']=p.text
                else:
                    if i%2==0: #홀수면 스탯 이름, 짝수면 스탯 값
                        key=p.text
                    elif i%2!=0:
                        detail_dict[key]=p.text
                    
        one_df=pd.DataFrame.from_dict(detail_dict, orient='index')
        total_df=pd.concat([total_df,one_df], axis=1)
    
    total_df=total_df.T
    total_df['url'] = url_list
    
    return total_df

In [19]:
driver.back()

In [23]:
click_by_class(5, 3)

In [24]:
urls = players_info(driver.page_source)
temp = StatDB(urls)
temp.head(3)

num of url :  500
['http://fifaonline4.inven.co.kr/dataninfo/player/?code=317000041', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=317173210', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=317173198', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=317184943', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=317009676']


,GK 다이빙,GK 반응속도,GK 위치선정,GK 킥,GK 핸들링,foot_left,foot_right_act,name,specialty,가로채기,...,크로스,킥,태클,패스,페널티 킥,프리킥,피지컬,핸들링,헤더,url
0,9,10,15,9,15,4,5,안드레스 이니에스타 루한,"특성예리한 감아차기, 플레이 메이커, 테크니컬 드리블러",62,...,72,NaN,54,80,67,66,54,NaN,51,http://fifaonline4.inven.co.kr/dataninfo/playe...
0,7,7,5,7,6,4,5,클라우디오 마르키시오,"특성유리몸, 화려한 개인기, 중거리 슛 선호",75,...,71,NaN,73,77,73,73,69,NaN,64,http://fifaonline4.inven.co.kr/dataninfo/playe...
0,7,7,7,7,7,5,3,다비데 아스토리,특성,80,...,34,NaN,79,50,27,19,71,NaN,72,http://fifaonline4.inven.co.kr/dataninfo/playe...


In [25]:
temp.to_csv("_17.csv")